In [86]:
import pandas as pd
import numpy as np
from copy import deepcopy
import math
from sklearn.model_selection import train_test_split

activities_to_classify = {
        'Get items from refrigerator/cabinets/drawers': 0,
        'Peel a cucumber': 1,
        'Clear cutting board': 2,
        'Slice a cucumber': 3,
        'Peel a potato': 4,
        'Slice a potato': 5,
        'Slice bread': 6,
        'Spread almond butter on a bread slice': 7,
        'Spread jelly on a bread slice': 8,
        'Open a jar of almond butter': 9,
        'Pour water from a pitcher into a glass': 10,
        'Clean a plate with a sponge': 11,
        'Clean a plate with a towel': 12,
        'Clean a pan with a sponge': 13,
        'Clean a pan with a towel': 14,
        'Get items from cabinets: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 15,
        'Set table: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 16,
        'Stack on table: 3 each large/small plates, bowls': 17,
        'Load dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 18,
        'Unload dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 19,
}

activities_to_verbs = {
        'Get items from refrigerator/cabinets/drawers': 'Get/Put',
        'Peel a cucumber': 'Slice',
        'Clear cutting board': 'Clean',
        'Slice a cucumber': 'Slice',
        'Peel a potato': 'Slice',
        'Slice a potato': 'Slice',
        'Slice bread': 'Slice',
        'Spread almond butter on a bread slice': 'Spread',
        'Spread jelly on a bread slice': 'Spread',
        'Open a jar of almond butter': 'Open/Close',
        'Pour water from a pitcher into a glass': 'Pour',
        'Clean a plate with a sponge': 'Clean',
        'Clean a plate with a towel': 'Clean',
        'Clean a pan with a sponge': 'Clean',
        'Clean a pan with a towel': 'Clean',
        'Get items from cabinets: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 'Get/Put',
        'Set table: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 'Stack',
        'Stack on table: 3 each large/small plates, bowls': 'Stack',
        'Load dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 'Load',
        'Unload dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 'Load'
}

verbs = {
    'Get/Put': 0,
    'Load': 1,
    'Slice': 2,
    'Spread': 3,
    'Open/Close': 4,
    'Pour': 5,
    'Clean': 6,
    'Stack': 7,
}

FPS = 30
ACTION_LENGTH = 5

def augmentation(data_frame):
    augmented_data = []

    for _, row in data_frame.iterrows():
        start_frame = row['start_frame']
        stop_frame = row['stop_frame']
        interval_size = FPS * ACTION_LENGTH 

        num_intervals = math.ceil((stop_frame - start_frame + 1) / interval_size)

        for i in range(num_intervals):
            new_start = start_frame + i * interval_size
            new_stop = min(new_start + interval_size - 1, stop_frame)  
            new_row = row.copy()
            new_row['start_frame'] = new_start
            new_row['stop_frame'] = new_stop
            new_row['start_timestamp'] = new_start/FPS
            new_row['stop_timestamp'] = new_stop/FPS
            augmented_data.append(new_row)

    augmented_dataframe = pd.DataFrame(augmented_data, columns=data_frame.columns)
    augmented_dataframe.reset_index(drop=True, inplace=True)

    return augmented_dataframe

def create_annotations_file(timestamp_file, annotations_file, type='train'):

    timestamps = pd.read_pickle(timestamp_file)
    timestamps = timestamps.drop(
        ['myo_left_timestamps', 'myo_right_timestamps', 'myo_left_readings', 'myo_right_readings'], axis=1)
    timestamps = timestamps.reset_index()

    start_timestamp = timestamps['end'].iloc[0]
    timestamps['start_frame'] = ((timestamps['start'] - start_timestamp) * FPS).astype(int)
    timestamps['stop_frame'] = ((timestamps['stop'] - start_timestamp) * FPS).astype(int)
    
    annotations = pd.read_pickle(annotations_file)
    annotations = annotations[annotations['file'] == 'S04_1.pkl']
    annotations = annotations.drop(['labels'], axis=1)
    
    complete_labels = pd.merge(timestamps, annotations, on='index', how='inner')
    
    complete_labels['uid'] = complete_labels['index']
    complete_labels['participant_id'] = 'S04'
    complete_labels['video_id'] = 'S04_1'
    complete_labels['verb'] = complete_labels['description_x']
    complete_labels['narration'] = complete_labels['description_x']
    complete_labels['verb_class'] = complete_labels['verb'].map(activities_to_classify)

    complete_labels = complete_labels[
        ['uid', 'participant_id', 'video_id', 'narration', 'start', 'stop', 'start_frame',
         'stop_frame', 'verb', 'verb_class']]
            
    complete_labels = complete_labels.sort_values(by='start_frame')
    
    #complete_labels = augmentation(complete_labels)
    complete_labels['uid'] = range(0, len(complete_labels))
    
    file_type = 'train' if type == 'train' else 'test'
    
    complete_labels.to_csv(f"S04_{file_type}_complete.csv", index=False)
    complete_labels.to_pickle(f"action_net_annotations_training/S04_{file_type}.pkl")
    
    reduced_labels = complete_labels.copy()
    
    reduced_labels['verb'] = reduced_labels['narration'].map(activities_to_verbs)
    reduced_labels['verb_class'] = reduced_labels['narration'].map(activities_to_verbs).map(verbs).astype(int)
    
    reduced_labels.to_csv(f"S04_{file_type}_reduced.csv", index=False)
    reduced_labels.to_pickle(f"action_net_annotations_extraction/S04_{file_type}.pkl")
    

def create_annotations_from_emg(emg_data, type='train'):
    calibration_val = 1.655239e+09
    
    emg_data = pd.read_pickle(emg_data)
    emg_data = emg_data[emg_data['file'] == 'S04_1.pkl']
    emg_data = emg_data.rename(columns={'file': 'video_id', 'description': 'narration'})
    emg_data['participant_id'] = 'S04'
    emg_data['video_id'] = 'S04_1'
    emg_data['start_frame'] = ((emg_data['start'] - calibration_val) * FPS).astype(int)
    emg_data['stop_frame'] = ((emg_data['stop'] - calibration_val) * FPS).astype(int)
    emg_data['verb'] = emg_data['narration']
    emg_data['verb_class'] = emg_data['narration'].map(activities_to_classify)
    emg_data = emg_data.drop(['emg_data'], axis = 1)
    emg_data = emg_data[
        ['uid', 'participant_id', 'video_id', 'narration', 'start', 'stop', 'start_frame',
         'stop_frame', 'verb', 'verb_class']]
    
    emg_data.to_pickle(f'action_net_annotations_training/S04_{"train" if type == "train" else "test"}.pkl')
    
    reduced_data = deepcopy(emg_data)
    reduced_data['verb'] = reduced_data['narration'].map(activities_to_verbs)
    reduced_data['verb_class'] = reduced_data['narration'].map(activities_to_verbs).map(verbs).astype(int)
    
    reduced_data.to_pickle(f'action_net_annotations_extraction/S04_{"train" if type == "train" else "test"}.pkl')

def create_reduced_annotations(complete_annotations, train=True):
    calibration_val = 1.655239e+09
    reduced_annotations = pd.read_pickle(complete_annotations)
    reduced_annotations['narration'] = reduced_annotations['description_x']
    reduced_annotations['video_id'] = "S04_1"
    reduced_annotations['participant_id'] = 'S04'
    reduced_annotations['verb'] = reduced_annotations['narration']
    reduced_annotations['verb_class'] = reduced_annotations['narration'].map(activities_to_classify)
    reduced_annotations['start_frame'] = ((reduced_annotations['start'] - calibration_val) * 30).astype(int)
    reduced_annotations['stop_frame'] = ((reduced_annotations['stop'] - calibration_val) * 30).astype(int)
    reduced_annotations = reduced_annotations[
        ['uid', 'participant_id', 'video_id', 'narration', 'start', 'stop', 'start_frame',
         'stop_frame', 'verb', 'verb_class']]
    reduced_annotations.to_pickle(f'action_net_annotations_training/S04_{"train" if train else "test"}.pkl')
    reduced_annotations['verb'] = reduced_annotations['narration'].map(activities_to_verbs)
    reduced_annotations['verb_class'] = reduced_annotations['narration'].map(activities_to_verbs).map(verbs)
    reduced_annotations.to_pickle(f'action_net_annotations_extraction/S04_{"train" if train else "test"}.pkl')

In [87]:
create_annotations_from_emg(emg_data='emg_preprocessed_train.pkl')
create_annotations_from_emg(emg_data='emg_preprocessed_test.pkl', type='test')

In [58]:
create_annotations_file(timestamp_file='S04_1.pkl', annotations_file='action-net/ActionNet_test.pkl', type='test')
create_annotations_file(timestamp_file='S04_1.pkl', annotations_file='action-net/ActionNet_train.pkl', type='train')


TypeError: issubclass() arg 1 must be a class

In [79]:
data = pd.read_pickle('S04_1.pkl')

data

,description,start,stop,myo_left_timestamps,myo_left_readings,myo_right_timestamps,myo_right_readings
0,calibration,1.655239e+09,1.655240e+09,[],[],[],[]
1,Get/replace items from refrigerator/cabinets/d...,1.655240e+09,1.655240e+09,"[1655239974.420555, 1655239974.432055, 1655239...","[[8, 7, 2, 2, 6, 5, 14, 2], [-4, 23, 19, 6, 16...","[1655239974.4195595, 1655239974.427058, 165523...","[[-2, -3, 2, 3, -8, -1, 1, -2], [3, 7, 8, -1, ..."
2,Peel a cucumber,1.655240e+09,1.655240e+09,"[1655240064.7444172, 1655240064.748419, 165524...","[[78, 57, 43, 7, 18, 18, 4, 16], [3, 0, 48, 6,...","[1655240064.7504191, 1655240064.76542, 1655240...","[[-18, 14, 26, -3, -15, -12, -26, 4], [-20, -2..."
3,Peel a cucumber,1.655240e+09,1.655240e+09,"[1655240130.5200577, 1655240130.531557, 165524...","[[-30, 20, 18, 1, 12, 23, 21, -10], [23, -15, ...","[1655240130.5190585, 1655240130.526558, 165524...","[[1, -21, 22, 14, 13, -24, -1, -12], [-58, -83..."
4,Peel a cucumber,1.655240e+09,1.655240e+09,"[1655240178.033217, 1655240178.037216, 1655240...","[[4, -19, -26, -63, -31, -36, -3, -1], [-30, -...","[1655240178.0472164, 1655240178.062216, 165524...","[[-4, 57, 49, 32, 34, 7, 7, 7], [-13, -80, -29..."
5,Clear cutting board,1.655240e+09,1.655240e+09,"[1655240238.6341186, 1655240238.645119, 165524...","[[-31, -13, 5, 33, 77, 20, 28, -20], [14, 16, ...","[1655240238.633119, 1655240238.636619, 1655240...","[[10, -9, -16, 0, -13, -2, -2, 2], [16, 15, 8,..."
6,Slice a cucumber,1.655240e+09,1.655240e+09,"[1655240291.109043, 1655240291.1125388, 165524...","[[7, -6, -2, -11, -22, -33, -17, 5], [-13, 3, ...","[1655240291.1150346, 1655240291.126035, 165524...","[[7, 13, 45, 14, -4, 1, 2, 4], [0, 2, -14, -10..."
7,Slice a cucumber,1.655240e+09,1.655240e+09,"[1655240357.383165, 1655240357.394165, 1655240...","[[-7, 12, 29, 4, -1, 18, -18, -24], [3, 14, 29...","[1655240357.378167, 1655240357.382166, 1655240...","[[3, 14, 9, 13, 11, 4, -11, -3], [-1, -12, -11..."
8,Slice a cucumber,1.655240e+09,1.655240e+09,"[1655240415.363077, 1655240415.3665743, 165524...","[[-14, -24, -14, -9, -8, -16, -47, -24], [-5, ...","[1655240415.3615725, 1655240415.365073, 165524...","[[-82, -25, -13, -28, -9, -75, 19, -1], [40, 1..."
9,Clear cutting board,1.655240e+09,1.655241e+09,"[1655240472.550571, 1655240472.5545695, 165524...","[[-8, 7, 20, 35, -11, 12, -11, 3], [-3, 17, 5,...","[1655240472.5575671, 1655240472.568567, 165524...","[[45, 31, 7, 14, 2, 3, -1, 12], [7, 6, -13, -1..."


In [82]:
data2 = pd.read_pickle('action-net/ActionNet_test.pkl')

data2 = data2[data2['file'] == 'S04_1.pkl'] 

data2

,index,file,description,labels
34,34,S04_1.pkl,Open/close a jar of almond butter,Open/Close
23,23,S04_1.pkl,Clear cutting board,Clear
26,26,S04_1.pkl,Spread almond butter on a bread slice,Spread
56,56,S04_1.pkl,"Set table: 3 each large/small plates, bowls, m...",Set
45,45,S04_1.pkl,Clean a plate with a towel,Clean
52,52,S04_1.pkl,Clean a pan with a towel,Clean
36,36,S04_1.pkl,Pour water from a pitcher into a glass,Pour
47,47,S04_1.pkl,Clean a plate with a towel,Clean


In [89]:
data2 = pd.read_pickle('action_net_annotations_training/S04_train.pkl')

data2

,uid,participant_id,video_id,narration,start,stop,start_frame,stop_frame,verb,verb_class
19,3159,S04,S04_1,Slice a potato,1.655241e+09,1.655241e+09,55773,56073,Slice a potato,5
20,2937,S04,S04_1,Peel a cucumber,1.655240e+09,1.655240e+09,36312,36612,Peel a cucumber,1
30,3419,S04,S04_1,Spread jelly on a bread slice,1.655241e+09,1.655241e+09,70589,70889,Spread jelly on a bread slice,8
61,3060,S04,S04_1,Peel a potato,1.655241e+09,1.655241e+09,48371,48671,Peel a potato,4
80,3137,S04,S04_1,Clear cutting board,1.655241e+09,1.655241e+09,54619,54919,Clear cutting board,2
...,...,...,...,...,...,...,...,...,...,...
8584,2911,S04,S04_1,Peel a cucumber,1.655240e+09,1.655240e+09,34376,34676,Peel a cucumber,1
8607,3112,S04,S04_1,Peel a potato,1.655241e+09,1.655241e+09,53076,53376,Peel a potato,4
8617,3439,S04,S04_1,Clear cutting board,1.655241e+09,1.655241e+09,71465,71765,Clear cutting board,2
8632,3201,S04,S04_1,Clear cutting board,1.655241e+09,1.655241e+09,57954,58254,Clear cutting board,2


In [91]:
data2 = pd.read_pickle('action_net_annotations_extraction/S04_test.pkl')

data2

,uid,participant_id,video_id,narration,start,stop,start_frame,stop_frame,verb,verb_class
16,262,S04,S04_1,"Set table: 3 each large/small plates, bowls, m...",1.655242e+09,1.655242e+09,91045,91345,Stack,7
17,277,S04,S04_1,"Set table: 3 each large/small plates, bowls, m...",1.655242e+09,1.655242e+09,92942,93242,Stack,7
26,238,S04,S04_1,Spread almond butter on a bread slice,1.655241e+09,1.655241e+09,67932,68232,Spread,3
27,237,S04,S04_1,Spread almond butter on a bread slice,1.655241e+09,1.655241e+09,67921,68221,Spread,3
30,278,S04,S04_1,"Set table: 3 each large/small plates, bowls, m...",1.655242e+09,1.655242e+09,93069,93369,Stack,7
...,...,...,...,...,...,...,...,...,...,...
836,276,S04,S04_1,"Set table: 3 each large/small plates, bowls, m...",1.655242e+09,1.655242e+09,92816,93116,Stack,7
847,221,S04,S04_1,Spread almond butter on a bread slice,1.655241e+09,1.655241e+09,67735,68035,Spread,3
871,263,S04,S04_1,"Set table: 3 each large/small plates, bowls, m...",1.655242e+09,1.655242e+09,91171,91471,Stack,7
903,206,S04,S04_1,Clear cutting board,1.655241e+09,1.655241e+09,63510,63810,Clean,6


In [92]:
data2 = pd.read_pickle('D4_train.pkl')

data2

,uid,participant_id,video_id,narration,verb,verb_class,start_timestamp,stop_timestamp,start_frame,stop_frame
132,132,P04,P04_01,Slice bread,Slice bread,11,1.655241e+09,1.655241e+09,58444,58594
202,15008,P04,P04_01,Clean a plate with a towel,Clean a plate with a towel,3,1.655242e+09,1.655242e+09,79048,79130
196,196,P04,P04_01,Get/replace items from refrigerator/cabinets/d...,Get/replace items from refrigerator/cabinets/d...,2,1.655242e+09,1.655242e+09,77229,77379
75,75,P04,P04_01,Get/replace items from refrigerator/cabinets/d...,Get/replace items from refrigerator/cabinets/d...,2,1.655241e+09,1.655241e+09,43822,43972
176,176,P04,P04_01,Get/replace items from refrigerator/cabinets/d...,Get/replace items from refrigerator/cabinets/d...,2,1.655241e+09,1.655241e+09,71307,71457
...,...,...,...,...,...,...,...,...,...,...
188,188,P04,P04_01,Get/replace items from refrigerator/cabinets/d...,Get/replace items from refrigerator/cabinets/d...,2,1.655242e+09,1.655242e+09,75789,75939
71,71,P04,P04_01,Get/replace items from refrigerator/cabinets/d...,Get/replace items from refrigerator/cabinets/d...,2,1.655241e+09,1.655241e+09,43102,43252
106,106,P04,P04_01,Clear cutting board,Clear cutting board,14,1.655241e+09,1.655241e+09,51243,51529
270,270,P04,P04_01,"Load dishwasher: 3 each large/small plates, bo...","Load dishwasher: 3 each large/small plates, bo...",9,1.655242e+09,1.655242e+09,95035,95185
